## Fine - Tuning a model to generate professional emails from notes

In [ ]:
# pip install --upgrade --quiet  openai python-dotenv

In [1]:
import json
import os
import time
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv("../keys.env")

assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please sign up for access to the OpenAI API and provide access token in keys.env file"

In [2]:
# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)

In [3]:
# Load the training data, you can create the training data with the generate_fine_tuning_dataset.py script
with open("fine_tuning_dataset.jsonl", "r") as f:
    training_data = [json.loads(line) for line in f]


In [4]:
# Let's infer the first sample
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=training_data[0]["messages"][:2]
)
print(f"Input: {training_data[0]['messages'][:2]}")
print("-"*100)
print(f"Output: {response.choices[0].message.content}")


Input: [{'role': 'system', 'content': 'You are a helpful assistant converting notes to professional emails.'}, {'role': 'user', 'content': 'feedback 4 mobile app\nlooked @ everything, here r my thots:\nReviewed current progress and milestones\nAligned on priorities for the next quarter\nDiscussed challenges and potential solutions\nlets discuss if needed'}]
----------------------------------------------------------------------------------------------------
Output: Subject: Feedback and Next Steps for Mobile App Development

Dear Team,

I hope this message finds you well. I have thoroughly reviewed our current progress and milestones for the mobile app development project and wanted to share my thoughts with you.

During our recent meeting, we had a productive discussion where we:

1. Reviewed the progress made so far and identified key milestones achieved.
2. Aligned on priorities for the upcoming quarter, ensuring clarity on our focus areas.
3. Discussed the challenges we are currentl

### Observation:

The model is able to handle the task, but it fails on the correct placeholder tags or isn't hitting the correct tone you prefer. Let's fix those issues by fine-tuning a model.


In [5]:
# Upload the training file
training_file = client.files.create(
    file=open("fine_tuning_dataset.jsonl", "rb"),
    purpose="fine-tune"
)

In [6]:

# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo"  # Base model to fine-tune
)


In [7]:
# Continuously check the status of the fine-tuning job
while True:
    job_status = client.fine_tuning.jobs.retrieve(job.id)
    print(f"Job status: {job_status.status}")

    if job_status.status in ['succeeded', 'failed']:
        break

    print("Waiting 60 seconds...")
    time.sleep(60)

if job_status.status == 'succeeded':
    print(f"Fine-tuning complete! You can now use model: {job_status.fine_tuned_model}")
else:
    print("Fine-tuning failed. Check the job status for more information.")

# Once the job is complete, you can use the fine-tuned model
# The fine-tuned model ID will be available in job_status.fine_tuned_model

Job status: validating_files
Waiting 60 seconds...
Job status: validating_files
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: running
Waiting 60 seconds...
Job status: succeeded
Fine-tuning complete! You can now use model: ft:gpt-3.5-turbo-0125:digits::B7waeTJO


In [8]:
# Now we can use the fine-tuned model to generate the email with training_data[0]['messages']
# Generally it is ad practise to test the model with a sample input from the training data,
# but we want to check the different outputs between the generic gpt-3.5-turbo and the fine-tuned model.

# Test the fine-tuned model with a sample input training_data[0]['messages']
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=training_data[0]['messages'][:2]
)

# Print the generated response
print("Generated email:")
print(completion.choices[0].message.content)

Generated email:
Subject: Feedback on mobile app

Body: Hello {name},

Thank you for sharing mobile app with me. I've reviewed it and would like to provide some constructive feedback.

Here are my main observations:
- Reviewed current progress and milestones
- Aligned on priorities for the next quarter
- Discussed challenges and potential solutions

I believe implementing these suggestions would further strengthen the mobile app. Please let me know if you'd like to discuss any of these points in more detail.

Kind regards,
{name}


In [9]:
# Now we can use the fine-tuned model to generate the email

# Test the fine-tuned model with a sample input
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=[
        {"role": "system", "content": "You are a helpful assistant converting notes to professional emails."},
        {"role": "user", "content": "schedule a meeting with John about the Q4 planning"}
    ]
)

# Print the generated response
print("Generated email:")
print(completion.choices[0].message.content)


Generated email:
Subject: Request for meeting

Body: Hi John,

I hope you're doing well. I'm reaching out because I'd like to discuss Q4 planning with you.

Could we please you set up a time to meet?
